In [1]:
from torchvision import datasets, transforms
import torch

# 데이터 세트
train_ds = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_ds  = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor(), download=True)

# 데이터 로더
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=True)

# CNN 분류 모델 만들기



- Conv 레이어를 이용해서 Feature Extraction
- Feature Extraction이 된 Feature Map을 이용해 추론하는 과정을 FCL로 구현

> Feature Extraction?

Feature Extraction은 CNN에서 데이터를 처리하고 의미 있는 정보를 추출하는 핵심 과정입니다. 

이를 통해 모델은 이미지나 데이터의 중요한 패턴을 학습하고, 불필요한 정보를 제거하여 더 효과적인 예측을 수행할 수 있습니다. 

CNN의 여러 층을 거치면서 저수준에서 고수준까지 다양한 특징이 추출되며, 이는 모델의 성능을 크게 향상시키는 역할을 합니다.



In [2]:
from torch import nn

class Mnist_CNN(nn.Module):

    def __init__(self):
        super().__init__()

        # 흑백이미지가 들어가기 때문에 입력 채널을 1로 설정
        # 필터의 shape : (32, 1, 3, 3)

        ######## Feature Extraction #########
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.relu_1 = nn.ReLU()

        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.relu_2 = nn.ReLU()

        self.max_pool_1 = nn.MaxPool2d(kernel_size=2)

        self.conv_3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu_3 = nn.ReLU()

        self.conv_4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.relu_4 = nn.ReLU()

        self.max_pool_2 = nn.MaxPool2d(kernel_size=2) # 여기 까지 오면 (N, 64, 7, 7)

        self.dropout = nn.Dropout(0.25)

        ######### Fully Conneceted Layer ########
        self.flatten = nn.Flatten() # 평탄화 시키면 (N, 64x7x7)

        self.linear_1 = nn.Linear(7*7*64, out_features=256)
        self.relu_5 = nn.ReLU()

        self.linear_2 = nn.Linear(256, 128)
        self.relu_6 = nn.ReLU()

        # 출력층
        self.out_layer = nn.Linear(128, 10)


    def forward(self, x):
        # Feature Extraction
        x = self.conv_1(x)
        x = self.relu_1(x)

        x = self.conv_2(x)
        x = self.relu_2(x)

        x = self.max_pool_1(x)

        x = self.conv_3(x)
        x = self.relu_3(x)

        x = self.conv_4(x)
        x = self.relu_4(x)

        x = self.max_pool_2(x)

        x = self.dropout(x)

        # FCL
        # 평탄화 부터
        x = self.flatten(x)

        x = self.linear_1(x)
        x = self.relu_5(x)

        x = self.linear_2(x)
        x = self.relu_6(x)

        y = self.out_layer(x)

        return y                                        

# 모델 생성

In [3]:
# MPS 지원 여부 확인
if torch.backends.mps.is_available():
    device = 'mps'  # MPS 사용
elif torch.cuda.is_available():
    device = 'cuda'  # CUDA 사용 가능 시
else:
    device = 'cpu'  # 그 외에는 CPU 사용

print(device)

mps


In [4]:
model = Mnist_CNN().to(device)
model

Mnist_CNN(
  (conv_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_1): ReLU()
  (conv_2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_2): ReLU()
  (max_pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_3): ReLU()
  (conv_4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_4): ReLU()
  (max_pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_1): Linear(in_features=3136, out_features=256, bias=True)
  (relu_5): ReLU()
  (linear_2): Linear(in_features=256, out_features=128, bias=True)
  (relu_6): ReLU()
  (out_layer): Linear(in_features=128, out_features=10, bias=True)
)

In [5]:
from torchinfo import summary
summary(model, input_size=(64, 1, 28, 28), device=device)

Layer (type:depth-idx)                   Output Shape              Param #
Mnist_CNN                                [64, 10]                  --
├─Conv2d: 1-1                            [64, 32, 28, 28]          320
├─ReLU: 1-2                              [64, 32, 28, 28]          --
├─Conv2d: 1-3                            [64, 32, 28, 28]          9,248
├─ReLU: 1-4                              [64, 32, 28, 28]          --
├─MaxPool2d: 1-5                         [64, 32, 14, 14]          --
├─Conv2d: 1-6                            [64, 64, 14, 14]          18,496
├─ReLU: 1-7                              [64, 64, 14, 14]          --
├─Conv2d: 1-8                            [64, 64, 14, 14]          36,928
├─ReLU: 1-9                              [64, 64, 14, 14]          --
├─MaxPool2d: 1-10                        [64, 64, 7, 7]            --
├─Dropout: 1-11                          [64, 64, 7, 7]            --
├─Flatten: 1-12                          [64, 3136]                --
├─L

# 훈련

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [7]:
# 훈련 루프 정의
def train_loop(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)

    # 모델을 훈련모드로 전환
    model.train()

    # 데이터 로더에서 데이터 꺼내기
    for batch, (X, y) in enumerate(dataloader):

        # 꺼낸 데이터를 gpu로 옮기기. 모델과 같은 디바이스 환경으로 설정
        X, y = X.to(device), y.to(device)
        
        # 예측 및 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파 수행(미분)
        optimizer.zero_grad() # 이전 배치에서의 기울기를 제거
        loss.backward() # 역전파 수행
        optimizer.step() # 파라미터 갱신

        # 배치가 100번 돌 때마다 화면에 출력
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Train Loss : {loss:>7f} [ {current:>5d} / {size:>5d} ]")

In [8]:
# 추론 루프 정의
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)

    test_loss, correct = 0, 0
    model.eval() # 모델을 추론 모드로 설정

    # 추론 과정에서는 기울기를 구하지 않음. required_grad=False
    with torch.no_grad():

        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    num_batches = len(dataloader)

    # Loss와 Accuracy 평균 계산
    test_loss /= num_batches
    correct /= size

    print(f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:>8f}\n")

In [9]:
# 학습 수행
epochs = 10

for t in range(epochs):

    print(f"Epoch {t+1}\n........................")
    train_loop(train_dl, model, loss_fn, optimizer)
    test_loop(test_dl, model, loss_fn)

print("Done!")

Epoch 1
........................
Train Loss : 2.294443 [     0 / 60000 ]
Train Loss : 1.096054 [  6400 / 60000 ]
Train Loss : 0.366841 [ 12800 / 60000 ]
Train Loss : 0.366278 [ 19200 / 60000 ]
Train Loss : 0.373609 [ 25600 / 60000 ]
Train Loss : 0.131739 [ 32000 / 60000 ]
Train Loss : 0.250502 [ 38400 / 60000 ]
Train Loss : 0.244062 [ 44800 / 60000 ]
Train Loss : 0.112270 [ 51200 / 60000 ]
Train Loss : 0.173976 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 96.4%, Avg Loss : 0.117974

Epoch 2
........................
Train Loss : 0.165161 [     0 / 60000 ]
Train Loss : 0.077277 [  6400 / 60000 ]
Train Loss : 0.132309 [ 12800 / 60000 ]
Train Loss : 0.061201 [ 19200 / 60000 ]
Train Loss : 0.120078 [ 25600 / 60000 ]
Train Loss : 0.149274 [ 32000 / 60000 ]
Train Loss : 0.074633 [ 38400 / 60000 ]
Train Loss : 0.079617 [ 44800 / 60000 ]
Train Loss : 0.074490 [ 51200 / 60000 ]
Train Loss : 0.057895 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 97.6%, Avg Loss : 0.074298

Epoch 3
..................

### 숙제

1. 파라미터 개수 계산


2. nn.Sequential로 구현
- 첫 번째 Sequential : Feature Extraction 과정 엮기
- 두 번째 Sequential : Fully Connetected Layer 과정 엮기